In [2]:
from class_dataset import ChestDataset
import pandas as pd
from keras.layers import Dense, Activation, Conv2D, MaxPool2D, Flatten, BatchNormalization, Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras import applications
from keras.applications import DenseNet121
from keras import models
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report,roc_curve,auc,accuracy_score
import os 
from glob import glob

In [27]:
csvfile = 'data_kaggle/Data_Entry_2017.csv'
df = pd.read_csv(csvfile)

data_dir = '../image/images/'
dataset = ChestDataset(data_dir,df)
df = dataset.reader
df = df[df.exists == True]
dataset = ChestDataset(data_dir,df)
df['path'] = dataset.image_path


train_list_file = '../multilabel exp/densechest_multilabel_train_list.txt'
test_list_file = '../multilabel exp/densechest_multilabel_test_list.txt'

df['Finding Labels'] = df['Finding Labels'].apply(lambda x: '' if x== 'No Finding' else x)
labels = list(df['Finding Labels'][~df['Finding Labels'].str.contains('\|')].unique())
labels.remove('')

for label in labels:
    df[label] = df['Finding Labels'].map(lambda x: 1 if label in x else 0)
df['disease_vec'] = df[labels].apply(lambda x: np.array(list(x)),axis=1)

with open(train_list_file,'r') as f:
    train_list = f.read().split('\n')
    
with open(test_list_file,'r') as f:
    test_list = f.read().split('\n')

/home/yair/Documents/ITC/Final/finalproject/class_dataset.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.reader['exists'] = self.exists


In [13]:
def load_json_model(json_file,weight_file):
    json_file = open(json_file, 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = models.model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights(weight_file)
    print("Loaded model from disk")
    return loaded_model

model_path = '../multilabel exp/densechest_multilabel.json'
weight_path = '../multilabel exp/densechest_multilabel.h5'
model = load_json_model(model_path,weight_path)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Loaded model from disk
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet121 (Model)          (None, 8, 8, 1024)        7037504   
_________________________________________________________________
flatten_1 (Flatten)          (None, 65536)             0         
_________________________________________________________________
activation_1 (Activation)    (None, 65536)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 14)                917518    
Total params: 7,955,022
Trainable params: 7,871,374
Non-trainable params: 83,648
_________________________________________________________________


In [25]:
def flow_from_dataframe(img_data_gen, in_df, path_col, y_col, **dflow_args):
    base_dir = os.path.dirname(in_df[path_col].values[0])
    print('## Ignore next message from keras, values are replaced anyways')
    df_gen = img_data_gen.flow_from_directory(base_dir, 
                                     class_mode = 'sparse',
                                    **dflow_args)
    df_gen.filenames = in_df[path_col].values
    df_gen.classes = np.stack(in_df[y_col].values)
    df_gen.samples = in_df.shape[0]
    df_gen.n = in_df.shape[0]
    df_gen._set_index_array()
    df_gen.directory = '' # since we have the full path
    print('Reinserting dataframe: {} images'.format(in_df.shape[0]))
    return df_gen

In [30]:
img_width,img_height = 256,256
validation_datagen = ImageDataGenerator(rescale=1./255)
val_batchsize = 10

df_test = df[df['Image Index'].isin(test_list)]

validation_generator = flow_from_dataframe(validation_datagen, df_test, 
                             path_col = 'path',
                            y_col = 'disease_vec', 
                            target_size = (img_width,img_height),
                             color_mode = 'rgb',
                            batch_size = val_batchsize)

## Ignore next message from keras, values are replaced anyways
Found 0 images belonging to 0 classes.
Reinserting dataframe: 7 images
